# Подготовка инструментов

Импорт необходимых инструментов

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import random
import joblib
from sklearn.preprocessing import LabelEncoder
import datetime
from datetime import date
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

Создадим перцептрон со случаными весами для имитации сложной функции

In [3]:
from copy import deepcopy

relu = np.vectorize(lambda x: x if x>0 else 0.)
sigm = lambda x: 1./(1+np.exp(-x))

vec_sizes = [45, 45, 45, 20, 15, 1]

def perceptron(xx, vec_sizes):
    
    xx_ext = xx
    generator = np.random.RandomState(43)
    
    for i, new_size in enumerate(vec_sizes):
        xx_ext = np.hstack([np.ones((xx_ext.shape[0], 1)), xx_ext])
        
        weights = generator.normal(0., 1, size=(xx_ext.shape[1], new_size))
        
        xx_ext = relu(xx_ext.dot(weights))
        
    return np.floor(xx_ext.reshape((-1,)))

# Генерация данных, построение модели

Сгенерируем нормально распределенные фичи и с помощью случайного 
перцептрона сымитируем зависимость целевой переменной. Сгенерированный 
датафрейм состоит из признаков, которые вошли в итоговую модель.

Предположим, что факторы feat_3, feat_8, feat_13, feat_27
являются защищенными характеристиками

In [5]:
# Генератор 
gen = np.random.RandomState(42)


# Генерация признаков
X = pd.DataFrame(data=gen.normal(50., 100., size=(30000, 30)),
                 columns=[f'feat_{i}' for i in range(30)],
                 index=[i for i in range(30000)])


# Имитация зависимости
y = pd.Series(data=1*(perceptron(X.values, [60, 32, 12, 5, 1])>0),
              index=[i for i in range(30000)],
              name='target'
             )

# Защищенные характеристики 
fair_factors = ['feat_3', 'feat_8', 'feat_13', 'feat_27']

# Остальные признаки
no_fair_factors = list(set(X.columns) - set(fair_factors))

Сгенеруем защищенные характеристики и "руками" внесем дискриминацию.
С помощью другого случайного перцептрона сымитируем зависимость 
одного защищенного фактора от признаков, не являющихся защищенными характеристиками

In [6]:
# Генерация
X['feat_3'] = gen.randint(18, 40, size=(30000,))

# Внесение "руками" дискриминации
X['feat_3'][y>0]=gen.randint(30, 41, size=(len(X['feat_3'][y>0]),))

# Генерация
X['feat_8'] = gen.choice([0, 1], size=(30000,), p=[0.4, 0.6])

# Внесение "руками" дискриминации
X['feat_8'][y>0]=gen.choice([0, 1], size=(len(X['feat_8'][y>0]),), p=[0.1, 0.9])

# Генерация
X['feat_13'] = gen.uniform(0, 10, size=(30000,))

# Имитация зависимости 
X['feat_27'] = perceptron(X[no_fair_factors].values, vec_sizes)


Разобьем данные

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_oos, y_train, y_oos = train_test_split(X, y, 
                                                  test_size=0.1, 
                                                  random_state=42,
                                                  stratify=y)

Построим итоговую модель

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

Качество итоговой модели

In [9]:
-1 + 2*roc_auc_score(y_oos, model.predict_proba(X_oos)[:, 1])

0.8616679910022158

# Пайплайн fairness

In [10]:
import sys
sys.path.append('fairness')

In [11]:
model

RandomForestClassifier()

In [12]:
X['feat_8'].unique()

array([0, 1])

In [34]:
from fairness_main import FairnessValidation

val = FairnessValidation(model,
                   train={'x': X_train, 'y': y_train},
                   oos={'x': X_oos, 'y': y_oos},
                   cat_features=list(X_train.columns[X_train.nunique() < 25]), 
                   protected_feats=fair_factors, n_bootstrap=64)

In [35]:
res = val.validate()

100%|██████████| 4/4 [01:02<00:00, 15.73s/it]


In [36]:
test_1, test_2, test_3, test_4 = res.values()
result_fair = pd.merge(pd.merge(pd.merge(test_1, test_2), test_3), test_4)

In [37]:
result_fair

,Защищенная характеристика,Левая граница CI_0.95 Gini,Правая граница CI_0.95 Gini,Защищенная характеристика не проверяется на дискриминацию,Минимальное относительное изменение прогнозного значения,Максимальное относительное изменение прогнозного значения,Результат теста 2,Изменение TPRD после перестановки значений,Изменение FPRD после перестановки значений,Результат теста 3,95.0%-ый доверительный интервал для угнетаемой группы,95.0%-ый доверительный интервал для привелегированной группы,99.0%-ый доверительный интервал для угнетаемой группы,99.0%-ый доверительный интервал для привелегированной группы,Результат теста 4
0,feat_3,0.100913,0.141981,no,-0.044643,0.013274,green,0.042054,0.000000,green,"(0.5600540533549749, 0.812239556924905)","невозможно расчитать, в группе один класс","(0.5204328391820143, 0.8518607710978656)","невозможно расчитать, в группе один класс",gray
1,feat_8,0.142903,0.266571,no,-0.563067,1.400000,yellow,0.043649,0.070248,green,"(0.8274208478527604, 0.9178483654489501)","(0.7897192055410251, 0.8448767600784837)","(0.8132136551668641, 0.9320555581348464)","(0.7810533256603468, 0.853542639959162)",green
2,feat_13,-0.013842,0.028165,no,0.000000,0.025424,green,0.006061,0.038462,green,"(0.7915459084246634, 0.9444680775893224)","(0.4440166945009294, 0.8578014873172524)","(0.767520094369152, 0.9684938916448338)","(0.3790063919038037, 0.9228117899143781)",green
3,feat_27,0.649177,0.693736,yes,-0.026786,0.000000,green,0.022901,0.000000,green,"(0.7003603968097872, 0.9687901247997511)","(0.8365909830542141, 0.9751138515513836)","(0.6581870300122356, 1.0109634915973027)","(0.8148274630941649, 0.9968773715114329)",green


In [18]:
result_fair.to_csv(f'Result_fairness_11111.csv', index = False, sep=';')
#result_fair.to_excel(f'Result_fairness_11111.xlsx', index = False)

# Пайплайн fairness для реальных данных 

In [ ]:
# import sys
# sys.path.append('fairness')

У вас должны быть:
1) выборки train и OOS
2) используемые признаки - здесь я взял их из модели разработчика
3) обученная вами модель valid_model с методом predict_proba
4) категориальные признаки - если не уверены, оставляйте в вызове метода FairnessValidation cat_features=[]

In [ ]:
fair_factors = ['sd_gender_cd',
'sd_age_yrs_frac_nv',
'sd_age_yrs_comp_nv',
'sd_client_valid_nflag',
'sd_stlmnt_type_cd',
'sd_russian_citizen_nflag',
'sd_resident_nflag',
'sd_sbrf_employee_nflag',
'dep_social_client_nflag',
'lne_coborrower_nflag',
'lne_loan_overdue_nflag',
'prl_employee_dzo_nflag',
'prl_social_disab_pension_nflag',
'seg_client_mp_segment_cd',
'seg_crd_pos_cat_group',
'seg_crd_trx_segm',
'seg_crd_trx_subsegm',
'seg_age_segment',
'sd_name_age_segment_cd',
'sd_age_mnth_comp_nv',
'sd_cb_staff_nflag',
'client_region']
feats_for_analyse = list(set(model.used_features) & set(fair_factors))

In [ ]:
from fairness_main import FairnessValidation

X_train_ = train[model.used_features].reset_index(drop=True)
X_oos_ = oos[model.used_features].reset_index(drop=True)
y_train_ = train['target'].reset_index(drop=True)
y_oos_ = oos['target'].reset_index(drop=True)
val = FairnessValidation(valid_model,
                   train={'x': X_train_, 'y': y_train_},
                   oos={'x': X_oos_, 'y': y_oos_},
                   cat_features=model.categorical_features, 
                   protected_feats=feats_for_analyse, n_bootstrap=64)

In [ ]:
res = val.validate()

In [ ]:
test_1, test_2, test_3, test_4 = res.values()
result_fair = pd.merge(pd.merge(pd.merge(test_1, test_2), test_3), test_4)

In [ ]:
result_fair

In [ ]:
#result_fair.to_csv(f'Result_fairness_108426.csv', index = False, sep=';')
result_fair.to_excel(f'Result_fairness_108426.xlsx', index = False)